In [1]:
import sys  
sys.path.insert(0, '../src')

import config

import data_processing as dp
import dataset as ds
import train_model as training

import SimpleNet as net

import torch
import torch.nn as nn
import torch.optim as optim

Device: cuda Name: Quadro P1000


In [2]:
# Filter on only 1 disease and image type = PA
disease_filter_list=['Atelectasis']
position_filter_list=['PA']

In [3]:
# Load data using data preprocessing module
df_data , labels      = dp.load_data(disease_filter_list=disease_filter_list,position_filter_list=position_filter_list)

In [4]:
# Make Multihot vector for data
df_data, dict_labels = dp.multi_hot_label(df_data, labels)

# Split into train, validate and test split
# Training set   = All training images * 80%
# Validation set = All training images * 20%
# Test set       = All test images

df_train, df_test    = dp.make_train_test_split(df_data)   
df_train, df_val     = dp.train_test_split(df_train)     

In [5]:
# Make an instance of SimpleNet model
# Any NN model can be instanciated
model = net.make_SimpleNet_model(len(labels))

In [6]:
# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()  # change to CrossEntropyLoss if  multiclass
optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)

In [7]:
# Create loaders for NIH data using dataset module
train_data_loader = ds.load_data(df_train, transform=ds.data_transforms['train'], shuffle=False, batch_size=config.TRAIN_BATCH_SIZE, num_workers=config.TRAIN_WORKERS)
val_data_loader   = ds.load_data(df_val  , transform=ds.data_transforms['test'] , shuffle=False, batch_size=config.VAL_BATCH_SIZE  , num_workers=config.VAL_WORKERS)

In [8]:
# Train the model here
# If multiple classes are selected while retreiving data, the training would train a multiclass model
# else a single class model is trained
print(f"Num Classes       : {len(labels)}")
print(f"Target Class List : {labels}\n")    

# Start the training (Single Class, 10 epochs)
trained_model = training.train_model(model, train_data_loader, val_data_loader, criterion, optimizer, num_epochs=10)

Num Classes       : 1
Target Class List : ['Atelectasis']

Training started
    Mode          : cuda
    Model type    : <class 'SimpleNet.SimpleNet'>
Epoch 1
.......... Epoch:  1 , Bacth:  10 , loss: 0.6394099056720733 Average batch time: 8.45171582698822 secs
.......... Epoch:  1 , Bacth:  20 , loss: 0.5461522206664086 Average batch time: 7.1053554773330685 secs
..
Epoch              : 1/10
Training Loss      : 0.5277532339096069
Validation Loss    : 0.2797892093658447
Validation Accuracy: 0.9230632393489282
Validation ROC     : 0.514570506529486
Epoch 2
.......... Epoch:  2 , Bacth:  10 , loss: 0.2986604362726212 Average batch time: 30.174270367622377 secs
.......... Epoch:  2 , Bacth:  20 , loss: 0.2951403558254242 Average batch time: 18.18978496789932 secs
..
Epoch              : 2/10
Training Loss      : 0.29457906701348047
Validation Loss    : 0.27234822511672974
Validation Accuracy: 0.9230632393489282
Validation ROC     : 0.47648137400481455
Epoch 3
.......... Epoch:  3 , Bacth

In [13]:
# Save the model
training.save_model(trained_model,num_epochs=10,num_classes=1)

Model Saved at: ../model/SimpleNet_10epoch_1classes.pth
